In [90]:
%config IPCompleter.greedy=True
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import glob, os
from IPython.display import display

In [91]:
# pre-processing data
def format(file):
    # skip reading all ball-related data
    df = pd.read_csv(file, sep=',', skiprows=1, warn_bad_lines=False, error_bad_lines=False, index_col=None, mangle_dupe_cols=True)
    # first column not required
    df.drop(columns=["info"], inplace=True)
    # df = pd.read_csv("C:\Users\alizo\OneDrive - University of Waterloo\Desktop\Ali\Universities\Data Science\Cricket ML\Source\1211660.csv")
    # data format is transposed
    df = df.transpose(copy=True)
    headers = df.iloc[0]
    df = df[1:]
    df.columns = headers
    # multiple columns with same names
    df.columns = pd.io.parsers.ParserBase({'names':df.columns})._maybe_dedup_names(df.columns)
    df.reset_index(inplace=True)
    df.rename(columns={ df.columns[0]: "team1", df.columns[1]: "team2", "umpire" : "umpire1", "umpire.1" : "umpire2"}, inplace=True)
    
    return df



In [92]:
df = pd.DataFrame()
path = "C:/Users/alizo/OneDrive - University of Waterloo/Desktop/Ali/Universities/Data Science/Cricket ML/Source"
all_files = glob.glob(os.path.join(path, "*.csv"))
for i in all_files:
    new_df = format(i)
    df = pd.concat([df, new_df], axis=0, ignore_index=True)
    
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   team1            142 non-null    object
 1   team2            142 non-null    object
 2   gender           142 non-null    object
 3   season           142 non-null    object
 4   date             142 non-null    object
 5   competition      142 non-null    object
 6   match_number     126 non-null    object
 7   venue            142 non-null    object
 8   city             96 non-null     object
 9   neutralvenue     23 non-null     object
 10  toss_winner      142 non-null    object
 11  toss_decision    142 non-null    object
 12  player_of_match  139 non-null    object
 13  umpire1          142 non-null    object
 14  umpire2          142 non-null    object
 15  reserve_umpire   117 non-null    object
 16  tv_umpire        139 non-null    object
 17  match_referee    134 non-null    ob

,team1,team2,gender,season,date,competition,match_number,venue,city,neutralvenue,...,umpire2,reserve_umpire,tv_umpire,match_referee,winner,winner_wickets,method,winner_runs,outcome,eliminator
0,Islamabad United,Peshawar Zalmi,male,2016/17,2017/02/09,Pakistan Super League,1,Dubai International Cricket Stadium,NaN,true,...,Shozab Raza,Asif Yaqoob,Rashid Riaz,RS Mahanama,Islamabad United,7,D/L,NaN,NaN,NaN
1,Lahore Qalandars,Quetta Gladiators,male,2016/17,2017/02/10,Pakistan Super League,2,Dubai International Cricket Stadium,NaN,true,...,Shozab Raza,Asif Yaqoob,Ahsan Raza,Mohammed Anees,Quetta Gladiators,NaN,NaN,8,NaN,NaN
2,Karachi Kings,Peshawar Zalmi,male,2016/17,2017/02/10,Pakistan Super League,3,Dubai International Cricket Stadium,NaN,true,...,Ahsan Raza,Shozab Raza,Asif Yaqoob,RS Mahanama,Peshawar Zalmi,7,NaN,NaN,NaN,NaN
3,Islamabad United,Lahore Qalandars,male,2016/17,2017/02/11,Pakistan Super League,4,Dubai International Cricket Stadium,NaN,true,...,Shozab Raza,Rashid Riaz,Ahsan Raza,RS Mahanama,Lahore Qalandars,6,NaN,NaN,NaN,NaN
4,Karachi Kings,Quetta Gladiators,male,2016/17,2017/02/11,Pakistan Super League,5,Dubai International Cricket Stadium,NaN,true,...,Rashid Riaz,Asif Yaqoob,Ahmed Shahab,Mohammed Anees,Quetta Gladiators,7,NaN,NaN,NaN,NaN


In [93]:
# city dictionary
city_map = {'Dubai International Cricket Stadium' : 'Dubai',
            'Sharjah Cricket Stadium' : 'Sharjah',
            'Gaddafi Stadium' : 'Lahore',
            'National Stadium' : 'Karachi',
            'Sheikh Zayed Stadium' : 'Abu Dhabi',
            'Multan Cricket Stadium' : 'Multan',
            'Rawalpindi Cricket Stadium' : 'Rawalpindi'}
df['city'] = df.apply(lambda row: city_map[row['venue']], axis=1)
# df.head()

In [81]:
df.toss_decision.unique()
# df.outcome.unique()
df[pd.isnull(df['neutralvenue'])]


,team1,team2,gender,season,date,competition,match_number,venue,city,neutralvenue,...,umpire2,reserve_umpire,tv_umpire,match_referee,winner,winner_wickets,method,winner_runs,outcome,eliminator
23,Peshawar Zalmi,Quetta Gladiators,male,2016/17,2017/03/05,Pakistan Super League,NaN,Gaddafi Stadium,Lahore,NaN,...,REJ Martinesz,NaN,Shozab Raza,RS Mahanama,Peshawar Zalmi,NaN,NaN,58,NaN,NaN
24,Peshawar Zalmi,Multan Sultans,male,2017/18,2018/02/22,Pakistan Super League,1,Dubai International Cricket Stadium,Dubai,NaN,...,RT Robinson,Rashid Riaz,REJ Martinesz,RS Mahanama,Multan Sultans,7,NaN,NaN,NaN,NaN
25,Karachi Kings,Quetta Gladiators,male,2017/18,2018/02/23,Pakistan Super League,2,Dubai International Cricket Stadium,Dubai,NaN,...,Rashid Riaz,Khalid Mahmood,Ahmed Shahab,Mohammed Anees,Karachi Kings,NaN,NaN,19,NaN,NaN
26,Multan Sultans,Lahore Qalandars,male,2017/18,2018/02/23,Pakistan Super League,3,Dubai International Cricket Stadium,Dubai,NaN,...,RT Robinson,Khalid Mahmood,Shozab Raza,RS Mahanama,Multan Sultans,NaN,NaN,43,NaN,NaN
27,Islamabad United,Peshawar Zalmi,male,2017/18,2018/02/24,Pakistan Super League,4,Dubai International Cricket Stadium,Dubai,NaN,...,Rashid Riaz,Khalid Mahmood,Ahmed Shahab,Mohammed Anees,Peshawar Zalmi,NaN,NaN,34,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Lahore Qalandars,Islamabad United,male,2015/16,2016/02/17,Pakistan Super League,20,Dubai International Cricket Stadium,Dubai,NaN,...,Aleem Dar,Ahmed Shahab,Shozab Raza,RS Mahanama,Islamabad United,5,NaN,NaN,NaN,NaN
138,Quetta Gladiators,Peshawar Zalmi,male,2015/16,2016/02/19,Pakistan Super League,NaN,Dubai International Cricket Stadium,Dubai,NaN,...,Aleem Dar,Ahmed Shahab,Ahsan Raza,RS Mahanama,Quetta Gladiators,NaN,NaN,1,NaN,NaN
139,Karachi Kings,Islamabad United,male,2015/16,2016/02/20,Pakistan Super League,NaN,Dubai International Cricket Stadium,Dubai,NaN,...,Ahsan Raza,Ahmed Shahab,JS Wilson,RS Mahanama,Islamabad United,9,NaN,NaN,NaN,NaN
140,Islamabad United,Peshawar Zalmi,male,2015/16,2016/02/21,Pakistan Super League,NaN,Dubai International Cricket Stadium,Dubai,NaN,...,Aleem Dar,Ahmed Shahab,Ahsan Raza,RS Mahanama,Islamabad United,NaN,NaN,50,NaN,NaN


In [94]:
# tie games
df['winner'] = df.apply(
    lambda row: row['eliminator'] if pd.isnull(row['winner']) and pd.notnull(row['eliminator']) else row['winner'],
    axis=1
)
# games decided by D/L (washed out) 
df['outcome'] = df.apply(
    lambda row: "D/L" if pd.isnull(row['outcome']) and (row['method'] == "D/L") else row['outcome'],
    axis=1
)
# neutralvenue column
df.neutralvenue.fillna(value=False, inplace=True)
df.neutralvenue.replace({'true' : True}, inplace=True)

# taking care of NA values
df.outcome.fillna("Result", inplace=True)
df.winner.fillna("Draw", inplace=True)    
df.drop(columns=["match_number", "eliminator", "method"], inplace=True)

In [95]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   team1            142 non-null    object
 1   team2            142 non-null    object
 2   gender           142 non-null    object
 3   season           142 non-null    object
 4   date             142 non-null    object
 5   competition      142 non-null    object
 6   venue            142 non-null    object
 7   city             142 non-null    object
 8   neutralvenue     142 non-null    bool  
 9   toss_winner      142 non-null    object
 10  toss_decision    142 non-null    object
 11  player_of_match  139 non-null    object
 12  umpire1          142 non-null    object
 13  umpire2          142 non-null    object
 14  reserve_umpire   117 non-null    object
 15  tv_umpire        139 non-null    object
 16  match_referee    134 non-null    object
 17  winner           142 non-null    ob

,team1,team2,gender,season,date,competition,venue,city,neutralvenue,toss_winner,...,player_of_match,umpire1,umpire2,reserve_umpire,tv_umpire,match_referee,winner,winner_wickets,winner_runs,outcome
count,142,142,142,142,142,142,142,142,142,142,...,139,142,142,117,139,134,142,89,49,142
unique,6,6,1,5,101,1,7,7,2,6,...,78,12,12,14,12,6,7,10,32,4
top,Islamabad United,Quetta Gladiators,male,2018/19,2017/02/18,Pakistan Super League,Dubai International Cricket Stadium,Dubai,False,Quetta Gladiators,...,Kamran Akmal,REJ Martinesz,Rashid Riaz,Khalid Mahmood,Shozab Raza,RS Mahanama,Peshawar Zalmi,5,1,Result
freq,38,44,142,34,2,142,60,60,119,32,...,7,36,28,28,38,66,30,23,5,135


In [96]:
# normalizing data
mapping = {"team1" : {"Islamabad United" : 1, "Quetta Gladiators" : 2, "Karachi Kings" : 3, "Peshawar Zalmi" : 4, "Multan Sultans" : 5, "Lahore Qalandars" : 6 },
           "team2" : {"Islamabad United" : 1, "Quetta Gladiators" : 2, "Karachi Kings" : 3, "Peshawar Zalmi" : 4, "Multan Sultans" : 5, "Lahore Qalandars" : 6 },
           "toss_winner" : {"Islamabad United" : 1, "Quetta Gladiators" : 2, "Karachi Kings" : 3, "Peshawar Zalmi" : 4, "Multan Sultans" : 5, "Lahore Qalandars" : 6 },
           "winner" : {"Islamabad United" : 1, "Quetta Gladiators" : 2, "Karachi Kings" : 3, "Peshawar Zalmi" : 4, "Multan Sultans" : 5, "Lahore Qalandars" : 6, "Draw" : 7}}

df.replace(mapping, inplace=True)

In [97]:
city_list = df['city']
city_map, city_index = pd.Series(city_list).factorize()
print(city_map)
print(city_index)

[0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 2 2 3 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 4 4 4 4 3 3 3 3 3 3 3 3 3 3 2 3 2 3 2 5 6 5 6 5 6 6 2 2 6 2 6 2
 6 2 2 3 3 3 2 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
Index(['Dubai', 'Sharjah', 'Lahore', 'Karachi', 'Abu Dhabi', 'Multan',
       'Rawalpindi'],
      dtype='object')


In [98]:
venue_list = df['venue']
venue_map, venue_index = pd.Series(venue_list).factorize()
print(venue_map)
print(venue_index)

[0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 2 2 3 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 4 4 4 4 3 3 3 3 3 3 3 3 3 3 2 3 2 3 2 5 6 5 6 5 6 6 2 2 6 2 6 2
 6 2 2 3 3 3 2 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
Index(['Dubai International Cricket Stadium', 'Sharjah Cricket Stadium',
       'Gaddafi Stadium', 'National Stadium', 'Sheikh Zayed Stadium',
       'Multan Cricket Stadium', 'Rawalpindi Cricket Stadium'],
      dtype='object')


In [99]:
toss_decision_list = df['toss_decision']
toss_decision_map, toss_decision_index = pd.Series(toss_decision_list).factorize()
print(toss_decision_map)
print(toss_decision_index)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0]
Index(['field', 'bat'], dtype='object')


In [100]:
# categorical to numerical
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
var = ['team1', 'team2', 'venue', 'neutralvenue', 'city', 'toss_winner', 'toss_decision', 'winner']
matches = df[var]
le = preprocessing.LabelEncoder()
for att in var:
    matches[att] = le.fit_transform(matches[att])
    


C:\Users\alizo\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [101]:
# function to run models
def run_model(model, data, attributes, labels, test_size, count, print_predictions):
    maximum = 0
    total = 0
    for i in range(count):
        x_train, x_test, y_train, y_test = train_test_split(data[attributes], data[labels], test_size=test_size) 
        model.fit(x_train, y_train.values.ravel())
        predictions = model.predict(x_test)
        if print_predictions: print(predictions)
        accuracy = metrics.accuracy_score(predictions, y_test)
        total = total + accuracy
        print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
        if maximum < accuracy: maximum = accuracy
    print("Average Accuracy: "+'%s' % '{0:.3%}'.format(total/count))
    print("Highest Accuracy: "+'%s' % '{0:.3%}'.format(maximum))

In [104]:
# create function uses test_train split
# run model multiple times and calculate average accuracy
# for KNN run muliple times with different neighbours from 1-7...


from sklearn.linear_model import LogisticRegression
# Logistic Regression

attributes = ['team1', 'team2', 'venue', 'neutralvenue', 'city', 'toss_winner', 'toss_decision']
labels = ['winner']


model = LogisticRegression(max_iter=1000)
run_model(model, matches, attributes, labels, 0.15, 10, False)

Accuracy : 13.636%
Accuracy : 18.182%
Accuracy : 18.182%
Accuracy : 18.182%
Accuracy : 31.818%
Accuracy : 36.364%
Accuracy : 27.273%
Accuracy : 27.273%
Accuracy : 22.727%
Accuracy : 13.636%
Average Accuracy: 22.727%
Highest Accuracy: 36.364%


In [107]:
# KNearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

attributes = ['team1', 'team2', 'venue', 'neutralvenue', 'city', 'toss_winner', 'toss_decision']
labels = ['winner']

model = KNeighborsClassifier(n_neighbors=2)

run_model(model, matches, attributes, labels, 0.15, 10, False)

Accuracy : 50.000%
Accuracy : 31.818%
Accuracy : 45.455%
Accuracy : 54.545%
Accuracy : 59.091%
Accuracy : 50.000%
Accuracy : 40.909%
Accuracy : 40.909%
Accuracy : 40.909%
Accuracy : 54.545%
Average Accuracy: 46.818%
Highest Accuracy: 59.091%


In [113]:
# Random Forest
# add graphviz to see trees
from sklearn import tree
attributes = ['team1', 'team2', 'venue', 'neutralvenue', 'city', 'toss_winner', 'toss_decision']
labels = ['winner']

print("Measure of Node Impurity: Gini")
model_gini = tree.DecisionTreeClassifier(criterion='gini', min_samples_leaf=3)
run_model(model_gini, matches, attributes, labels, 0.15, 5, False)
print(model_gini.feature_importances_)
print(model_gini.tree_)

print("Measure of Node Impurity: Entropy")
model_entropy = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)
run_model(model_entropy, matches, attributes, labels, 0.15, 5, False)
print(model_entropy.feature_importances_)
print(model_entropy.tree_)


Measure of Node Impurity: Gini
Accuracy : 40.909%
Accuracy : 40.909%
Accuracy : 54.545%
Accuracy : 50.000%
Accuracy : 68.182%
Average Accuracy: 50.909%
Highest Accuracy: 68.182%
[0.12146761 0.3395563  0.04580645 0.02457605 0.1466661  0.32064478
 0.0012827 ]
Measure of Node Impurity: Entropy
Accuracy : 50.000%
Accuracy : 36.364%
Accuracy : 50.000%
Accuracy : 63.636%
Accuracy : 50.000%
Average Accuracy: 50.000%
Highest Accuracy: 63.636%
[0.22283162 0.38817065 0.04166651 0.04365961 0.08277747 0.21923432
 0.00165982]


In [ ]:
# Gradient Boosted Classification

In [14]:
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   team1            142 non-null    int64 
 1   team2            142 non-null    int64 
 2   gender           142 non-null    object
 3   season           142 non-null    object
 4   date             142 non-null    object
 5   competition      142 non-null    object
 6   venue            142 non-null    int32 
 7   city             142 non-null    int32 
 8   neutralvenue     23 non-null     object
 9   toss_winner      142 non-null    int64 
 10  toss_decision    142 non-null    int32 
 11  player_of_match  139 non-null    object
 12  umpire1          142 non-null    object
 13  umpire2          142 non-null    object
 14  reserve_umpire   117 non-null    object
 15  tv_umpire        139 non-null    object
 16  match_referee    134 non-null    object
 17  winner           142 non-null    in

,team1,team2,gender,season,date,competition,venue,city,neutralvenue,toss_winner,...,player_of_match,umpire1,umpire2,reserve_umpire,tv_umpire,match_referee,winner,winner_wickets,winner_runs,outcome
0,0,3,male,2016/17,2017/02/09,Pakistan Super League,0,1,true,0,...,BJ Haddin,Ahsan Raza,Shozab Raza,Asif Yaqoob,Rashid Riaz,RS Mahanama,0,7,NaN,D/L
1,5,1,male,2016/17,2017/02/10,Pakistan Super League,0,1,true,5,...,Hassan Khan,Rashid Riaz,Shozab Raza,Asif Yaqoob,Ahsan Raza,Mohammed Anees,1,NaN,8,Result
2,2,3,male,2016/17,2017/02/10,Pakistan Super League,0,1,true,3,...,EJG Morgan,Ahmed Shahab,Ahsan Raza,Shozab Raza,Asif Yaqoob,RS Mahanama,3,7,NaN,Result
3,0,5,male,2016/17,2017/02/11,Pakistan Super League,0,1,true,5,...,JJ Roy,Asif Yaqoob,Shozab Raza,Rashid Riaz,Ahsan Raza,RS Mahanama,5,6,NaN,Result
4,2,1,male,2016/17,2017/02/11,Pakistan Super League,0,1,true,1,...,RR Rossouw,Aleem Dar,Rashid Riaz,Asif Yaqoob,Ahmed Shahab,Mohammed Anees,1,7,NaN,Result
